# Навчання агентів для Дилеми в'язня використовуючи Трансформери


Цей ноутбук демонструє використання архітектури трансформерів (Decision Transformer) для навчання агента 
стратегії в Ітерованій Дилемі В'язня (Iterated Prisoner's Dilemma, IPD).

## Що ми будемо робити:
1. Налаштуємо середовище IPD
2. Зберемо дані з успішних партій для навчання трансформера
3. Реалізуємо модель Decision Transformer
4. Оцінимо продуктивність агента проти різних класичних стратегій
5. Проаналізуємо навчену стратегію та її ефективність

## Очікувані результати:
- Порівняння ефективності трансформер-агента з іншими підходами
- Розуміння того, як трансформери можуть навчатися з траєкторій
- Візуалізація процесу навчання та результатів


In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
from IPython.display import display
from torch.utils.data import Dataset, DataLoader

# Додаємо шлях до кореневої директорії проекту для можливості імпорту
sys.path.append(os.path.abspath('..'))

# Імпортуємо наше середовище IPD та стратегії
from env import IPDEnv, TitForTat, AlwaysCooperate, AlwaysDefect, RandomStrategy, PavlovStrategy

# Налаштування стилю візуалізації
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# Перевіряємо доступність GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Використовуємо пристрій: {device}')